# Pose Detection Using Movenet

## 1. Introduction
This notebook uses [Movenet model from TensorFlow Hub](https://www.tensorflow.org/hub/tutorials/movenet).

### 1.1. Pre-requisites
* Download [lightning](https://tfhub.dev/google/movenet/singlepose/lightning/4) version of the movenet model from the TF hub.
* Good to enable GPU as the inference is a little faster on it.

## 2. Extract And Save Poses

In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import time
from pathlib import Path
import pandas as pd
import cProfile
    
# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

tf.config.list_physical_devices()
# tf.debugging.set_log_device_placement(True)
# tf.config.set_visible_devices([], 'GPU')

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

import configuration
import customdataset

config = configuration.Configuration()
face_config = configuration.FaceConfig(config)
pose_config = configuration.PoseConfig(config)
    

GPU found


In [21]:
# def test_gpu_with_cifar():
#     cifar = tf.keras.datasets.cifar100
#     (x_train, y_train), (x_test, y_test) = cifar.load_data()
#     model = tf.keras.applications.ResNet50(
#         include_top=True,
#         weights=None,
#         input_shape=(32, 32, 3),
#         classes=100,)

#     loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#     model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
#     model.fit(x_train, y_train, epochs=5, batch_size=64)

In [22]:
# # Dictionary that maps from joint names to keypoint indices.
# KEYPOINT_DICT = {
#     'nose': 0,
#     'left_eye': 1,
#     'right_eye': 2,
#     'left_ear': 3,
#     'right_ear': 4,
#     'left_shoulder': 5,
#     'right_shoulder': 6,
#     'left_elbow': 7,
#     'right_elbow': 8,
#     'left_wrist': 9,
#     'right_wrist': 10,
#     'left_hip': 11,
#     'right_hip': 12,
#     'left_knee': 13,
#     'right_knee': 14,
#     'left_ankle': 15,
#     'right_ankle': 16
# }

# # Maps bones to a matplotlib color name.
# KEYPOINT_EDGE_INDS_TO_COLOR = {
#     (0, 1): 'm',
#     (0, 2): 'c',
#     (1, 3): 'm',
#     (2, 4): 'c',
#     (0, 5): 'm',
#     (0, 6): 'c',
#     (5, 7): 'm',
#     (7, 9): 'm',
#     (6, 8): 'c',
#     (8, 10): 'c',
#     (5, 6): 'y',
#     (5, 11): 'm',
#     (6, 12): 'c',
#     (11, 12): 'y',
#     (11, 13): 'm',
#     (13, 15): 'm',
#     (12, 14): 'c',
#     (14, 16): 'c'
# }

# def _keypoints_and_edges_for_display(keypoints_with_scores,
#                                      height,
#                                      width,
#                                      keypoint_threshold=0.11):
#     """Returns high confidence keypoints and edges for visualization.

#     Args:
#         keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
#           the keypoint coordinates and scores returned from the MoveNet model.
#     height: height of the image in pixels.
#     width: width of the image in pixels.
#         keypoint_threshold: minimum confidence score for a keypoint to be
#         visualized.

#     Returns:
#         A (keypoints_xy, edges_xy, edge_colors) containing:
#           * the coordinates of all keypoints of all detected entities;
#           * the coordinates of all skeleton edges of all detected entities;
#           * the colors in which the edges should be plotted.
#     """
#     keypoints_all = []
#     keypoint_edges_all = []
#     edge_colors = []
#     num_instances, _, _, _ = keypoints_with_scores.shape
#     for idx in range(num_instances):
#         kpts_x = keypoints_with_scores[0, idx, :, 1]
#         kpts_y = keypoints_with_scores[0, idx, :, 0]
#         kpts_scores = keypoints_with_scores[0, idx, :, 2]
#         kpts_absolute_xy = np.stack(
#             [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
#         kpts_above_thresh_absolute = kpts_absolute_xy[
#             kpts_scores > keypoint_threshold, :]
#         keypoints_all.append(kpts_above_thresh_absolute)

#         for edge_pair, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
#             if (kpts_scores[edge_pair[0]] > keypoint_threshold and
#               kpts_scores[edge_pair[1]] > keypoint_threshold):
#                 x_start = kpts_absolute_xy[edge_pair[0], 0]
#                 y_start = kpts_absolute_xy[edge_pair[0], 1]
#                 x_end = kpts_absolute_xy[edge_pair[1], 0]
#                 y_end = kpts_absolute_xy[edge_pair[1], 1]
#                 line_seg = np.array([[x_start, y_start], [x_end, y_end]])
#                 keypoint_edges_all.append(line_seg)
#                 edge_colors.append(color)
#     if keypoints_all:
#         keypoints_xy = np.concatenate(keypoints_all, axis=0)
#     else:
#         keypoints_xy = np.zeros((0, 17, 2))

#     if keypoint_edges_all:
#         edges_xy = np.stack(keypoint_edges_all, axis=0)
#     else:
#         edges_xy = np.zeros((0, 2, 2))
#     return keypoints_xy, edges_xy, edge_colors


# def draw_prediction_on_image(
#     image, keypoints_with_scores, crop_region=None, close_figure=False,
#     output_image_height=None):
#     """Draws the keypoint predictions on image.

#         Args:
#         image: A numpy array with shape [height, width, channel] representing the
#           pixel values of the input image.
#         keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
#           the keypoint coordinates and scores returned from the MoveNet model.
#         crop_region: A dictionary that defines the coordinates of the bounding box
#           of the crop region in normalized coordinates (see the init_crop_region
#           function below for more detail). If provided, this function will also
#           draw the bounding box on the image.
#         output_image_height: An integer indicating the height of the output image.
#           Note that the image aspect ratio will be the same as the input image.

#       Returns:
#         A numpy array with shape [out_height, out_width, channel] representing the
#         image overlaid with keypoint predictions.
#       """
#     height, width, channel = image.shape
#     aspect_ratio = float(width) / height
#     fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
#     # To remove the huge white borders
#     fig.tight_layout(pad=0)
#     ax.margins(0)
#     ax.set_yticklabels([])
#     ax.set_xticklabels([])
#     plt.axis('off')

#     im = ax.imshow(image)
#     line_segments = LineCollection([], linewidths=(4), linestyle='solid')
#     ax.add_collection(line_segments)
#     # Turn off tick labels
#     scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

#     (keypoint_locs, keypoint_edges,
#        edge_colors) = _keypoints_and_edges_for_display(
#        keypoints_with_scores, height, width)

#     line_segments.set_segments(keypoint_edges)
#     line_segments.set_color(edge_colors)
#     if keypoint_edges.shape[0]:
#         line_segments.set_segments(keypoint_edges)
#         line_segments.set_color(edge_colors)
#     if keypoint_locs.shape[0]:
#         scat.set_offsets(keypoint_locs)

#     if crop_region is not None:
#         xmin = max(crop_region['x_min'] * width, 0.0)
#         ymin = max(crop_region['y_min'] * height, 0.0)
#         rec_width = min(crop_region['x_max'], 0.99) * width - xmin
#         rec_height = min(crop_region['y_max'], 0.99) * height - ymin
#         rect = patches.Rectangle(
#             (xmin,ymin),rec_width,rec_height,
#             linewidth=1,edgecolor='b',facecolor='none')
#         ax.add_patch(rect)

#     fig.canvas.draw()
#     image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
#     image_from_plot = image_from_plot.reshape(
#       fig.canvas.get_width_height()[::-1] + (3,))
#     plt.close(fig)
#     if output_image_height is not None:
#         output_image_width = int(output_image_height / height * width)
#         image_from_plot = cv2.resize(
#             image_from_plot, dsize=(output_image_width, output_image_height),
#              interpolation=cv2.INTER_CUBIC)
#     return image_from_plot


In [23]:
#@title Cropping Algorithm

# Confidence score to determine whether a keypoint prediction is reliable.
MIN_CROP_KEYPOINT_SCORE = 0.2

KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# Maps bones to a matplotlib color name.
KEYPOINT_EDGE_INDS_TO_COLOR = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def _keypoints_and_edges_for_display(keypoints_with_scores,
                                     height,
                                     width,
                                     keypoint_threshold=0.11):
    """Returns high confidence keypoints and edges for visualization.

    Args:
        keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
          the keypoint coordinates and scores returned from the MoveNet model.
    height: height of the image in pixels.
    width: width of the image in pixels.
        keypoint_threshold: minimum confidence score for a keypoint to be
        visualized.

    Returns:
        A (keypoints_xy, edges_xy, edge_colors) containing:
          * the coordinates of all keypoints of all detected entities;
          * the coordinates of all skeleton edges of all detected entities;
          * the colors in which the edges should be plotted.
    """
    keypoints_all = []
    keypoint_edges_all = []
    edge_colors = []
    num_instances, _, _, _ = keypoints_with_scores.shape
    for idx in range(num_instances):
        kpts_x = keypoints_with_scores[0, idx, :, 1]
        kpts_y = keypoints_with_scores[0, idx, :, 0]
        kpts_scores = keypoints_with_scores[0, idx, :, 2]
        kpts_absolute_xy = np.stack(
            [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
        kpts_above_thresh_absolute = kpts_absolute_xy[
            kpts_scores > keypoint_threshold, :]
        keypoints_all.append(kpts_above_thresh_absolute)

        for edge_pair, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
            if (kpts_scores[edge_pair[0]] > keypoint_threshold and
              kpts_scores[edge_pair[1]] > keypoint_threshold):
                x_start = kpts_absolute_xy[edge_pair[0], 0]
                y_start = kpts_absolute_xy[edge_pair[0], 1]
                x_end = kpts_absolute_xy[edge_pair[1], 0]
                y_end = kpts_absolute_xy[edge_pair[1], 1]
                line_seg = np.array([[x_start, y_start], [x_end, y_end]])
                keypoint_edges_all.append(line_seg)
                edge_colors.append(color)
    if keypoints_all:
        keypoints_xy = np.concatenate(keypoints_all, axis=0)
    else:
        keypoints_xy = np.zeros((0, 17, 2))

    if keypoint_edges_all:
        edges_xy = np.stack(keypoint_edges_all, axis=0)
    else:
        edges_xy = np.zeros((0, 2, 2))
    return keypoints_xy, edges_xy, edge_colors


def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
    """Draws the keypoint predictions on image.

        Args:
        image: A numpy array with shape [height, width, channel] representing the
          pixel values of the input image.
        keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
          the keypoint coordinates and scores returned from the MoveNet model.
        crop_region: A dictionary that defines the coordinates of the bounding box
          of the crop region in normalized coordinates (see the init_crop_region
          function below for more detail). If provided, this function will also
          draw the bounding box on the image.
        output_image_height: An integer indicating the height of the output image.
          Note that the image aspect ratio will be the same as the input image.

      Returns:
        A numpy array with shape [out_height, out_width, channel] representing the
        image overlaid with keypoint predictions.
      """
    height, width, channel = image.shape
    aspect_ratio = float(width) / height
    fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
    # To remove the huge white borders
    fig.tight_layout(pad=0)
    ax.margins(0)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    plt.axis('off')

    im = ax.imshow(image)
    line_segments = LineCollection([], linewidths=(4), linestyle='solid')
    ax.add_collection(line_segments)
    # Turn off tick labels
    scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

    (keypoint_locs, keypoint_edges,
       edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
    if keypoint_edges.shape[0]:
        line_segments.set_segments(keypoint_edges)
        line_segments.set_color(edge_colors)
    if keypoint_locs.shape[0]:
        scat.set_offsets(keypoint_locs)

    if crop_region is not None:
        xmin = max(crop_region['x_min'] * width, 0.0)
        ymin = max(crop_region['y_min'] * height, 0.0)
        rec_width = min(crop_region['x_max'], 0.99) * width - xmin
        rec_height = min(crop_region['y_max'], 0.99) * height - ymin
        rect = patches.Rectangle(
            (xmin,ymin),rec_width,rec_height,
            linewidth=1,edgecolor='b',facecolor='none')
        ax.add_patch(rect)

    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)
    if output_image_height is not None:
        output_image_width = int(output_image_height / height * width)
        image_from_plot = cv2.resize(
            image_from_plot, dsize=(output_image_width, output_image_height),
             interpolation=cv2.INTER_CUBIC)
    return image_from_plot

def torso_visible(keypoints):
    """Checks whether there are enough torso keypoints.

    This function checks whether the model is confident at predicting one of the
    shoulders/hips which is required to determine a good crop region.
    """
    return ((keypoints[0, 0, KEYPOINT_DICT['left_hip'], 2] >
             MIN_CROP_KEYPOINT_SCORE or
            keypoints[0, 0, KEYPOINT_DICT['right_hip'], 2] >
             MIN_CROP_KEYPOINT_SCORE) and
            (keypoints[0, 0, KEYPOINT_DICT['left_shoulder'], 2] >
             MIN_CROP_KEYPOINT_SCORE or
            keypoints[0, 0, KEYPOINT_DICT['right_shoulder'], 2] >
             MIN_CROP_KEYPOINT_SCORE))

def determine_torso_and_body_range(
    keypoints, target_keypoints, center_y, center_x):
    """Calculates the maximum distance from each keypoints to the center location.

    The function returns the maximum distances from the two sets of keypoints:
    full 17 keypoints and 4 torso keypoints. The returned information will be
    used to determine the crop size. See determineCropRegion for more detail.
    """
    torso_joints = ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']
    max_torso_yrange = 0.0
    max_torso_xrange = 0.0
    for joint in torso_joints:
        dist_y = abs(center_y - target_keypoints[joint][0])
        dist_x = abs(center_x - target_keypoints[joint][1])
        if dist_y > max_torso_yrange:
            max_torso_yrange = dist_y
        if dist_x > max_torso_xrange:
            max_torso_xrange = dist_x

    max_body_yrange = 0.0
    max_body_xrange = 0.0
    for joint in KEYPOINT_DICT.keys():
        if keypoints[0, 0, KEYPOINT_DICT[joint], 2] < MIN_CROP_KEYPOINT_SCORE:
            continue
        dist_y = abs(center_y - target_keypoints[joint][0]);
        dist_x = abs(center_x - target_keypoints[joint][1]);
        if dist_y > max_body_yrange:
            max_body_yrange = dist_y

        if dist_x > max_body_xrange:
            max_body_xrange = dist_x

    return [max_torso_yrange, max_torso_xrange, max_body_yrange, max_body_xrange]

# def crop_and_resize(image, crop_region, crop_size):
#     """Crops and resize the image to prepare for the model input."""
#     boxes=[[crop_region['y_min'], crop_region['x_min'],
#             crop_region['y_max'], crop_region['x_max']]]
#     output_image = tf.image.crop_and_resize(
#         image, box_indices=[0], boxes=boxes, crop_size=crop_size)
#     return output_image

# def run_inference(movenet, image, crop_region, crop_size):
#     """Runs model inferece on the cropped region.

#     The function runs the model inference on the cropped region and updates the
#     model output to the original image coordinate system.
#     """
#     image_height, image_width, _ = image.shape
#     input_image = crop_and_resize(
#       tf.expand_dims(image, axis=0), crop_region, crop_size=crop_size)
#     # Run model inference.
#     keypoints_with_scores = movenet(input_image)
#     # Update the coordinates.
#     for idx in range(17):
#         keypoints_with_scores[0, 0, idx, 0] = (
#               crop_region['y_min'] * image_height +
#               crop_region['height'] * image_height *
#               keypoints_with_scores[0, 0, idx, 0]) / image_height
#         keypoints_with_scores[0, 0, idx, 1] = (
#               crop_region['x_min'] * image_width +
#               crop_region['width'] * image_width *
#               keypoints_with_scores[0, 0, idx, 1]) / image_width
#     return keypoints_with_scores

In [24]:
model_name = "movenet_thunder" #@param ["movenet_lightning", "movenet_thunder", "movenet_lightning_f16.tflite", "movenet_thunder_f16.tflite", "movenet_lightning_int8.tflite", "movenet_thunder_int8.tflite"]
model_name = "movenet_thunder"

# Lightning
if "movenet_lightning" in model_name:
    input_size = 192
    model_file = 'movenet_singlepose_lightning_4'        
elif "movenet_thunder" in model_name:
    model_file = 'movenet_singlepose_thunder_4'
    input_size = 256
else:
    raise ValueError("Unsupported model name: %s" % model_name)

model = tf.saved_model.load(model_file)

    
def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    
    # model = tf.saved_model.load(model_file)

    infer = model.signatures["serving_default"]

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = infer(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints_with_scores = outputs['output_0'].numpy()
    return keypoints_with_scores

In [25]:
def get_bounding_box(keypoints_with_scores, height, width, input_size, keypoint_threshold=0.0):
    kpts_x = keypoints_with_scores[0, 0, :, 1]
    kpts_y = keypoints_with_scores[0, 0, :, 0]
    kpts_scores = keypoints_with_scores[0, 0, :, 2]
    kpts_absolute_xy = np.stack(
        [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
    kpts_above_thresh_absolute = kpts_absolute_xy[kpts_scores > keypoint_threshold, :]
    kpts_x = kpts_above_thresh_absolute[:, 1]
    kpts_y = kpts_above_thresh_absolute[:, 0]
    x_min = kpts_x.min() - 10 
    y_min = kpts_y.min() - 10
    x_max = kpts_x.max() + 10
    y_max = kpts_y.max() + 10

    return {
      'y_min': y_min,
      'x_min': x_min,
      'y_max': y_max,
      'x_max': x_max,
      'height': y_max - y_min,
      'width': x_max - x_min
    }
    
    
def inference(image_path):
    start = time.time()
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image)
    image_height, image_width, _ = image.shape
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

    # Run model inference.
    keypoints_with_scores = movenet(input_image)
    # print(f'keypoints_with_scores: {keypoints_with_scores}')
    
    # Get ideal crop region.
    # crop_region = get_bounding_box(
      # keypoints_with_scores, image_height, image_width, input_size)

    # Visualize the predictions with image.
    display_image = tf.expand_dims(image, axis=0)
    display_image = tf.cast(tf.image.resize_with_pad(
        display_image, 1280, 1280), dtype=tf.int32)
    output_overlay = draw_prediction_on_image(
        np.squeeze(display_image.numpy(), axis=0), keypoints_with_scores)

    fig = plt.figure(figsize=(5, 5))
    plt.imshow(output_overlay)
    _ = plt.axis('off')
    # plot_bounding_box(crop_region)
    plt.show()
    
    # Show just the pose
    
    skeleton = draw_prediction_on_image(
        np.squeeze(np.zeros(display_image.shape), axis=0), keypoints_with_scores)
    fig = plt.figure(figsize=(5, 5))
    plt.imshow(skeleton)
    _ = plt.axis('off')
    # plot_bounding_box(crop_region)
    plt.show()

    end = time.time()
    print(f'Time taken: {end - start}')
    



In [26]:
# inference('input_image.jpeg')
# inference('img_13.jpg')
# inference('img_24.jpg')
# inference('img_28.jpg')
# inference('img_36.jpg')


In [27]:
%%time



CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 11 µs


In [29]:
%%time
# extract_poses(config, pose_config.FEATURES_FOLDER_FULL, pose_config.SUMMARY_NAME, limit=800)
extract_poses(config, f'{config.OUTPUT_FOLDER}/tt', pose_config.SUMMARY_NAME, limit=5)
# cProfile.run(
# "extract_poses(config, f'{config.OUTPUT_FOLDER}/tt', pose_config.SUMMARY_NAME, limit=5)", f'{config.OUTPUT_FOLDER}/mtcnn_stats_model_load')

Processing c0 files...
Processed 0:0 files...
Extracted from 0 files...
Keypoints saved to /Users/rasentha/mids/w281/project/w281-fall2022-section2-team3/output/tt/c0/img_94962_keypoints.pt
Keypoints saved to /Users/rasentha/mids/w281/project/w281-fall2022-section2-team3/output/tt/c0/img_50920_keypoints.pt
Keypoints saved to /Users/rasentha/mids/w281/project/w281-fall2022-section2-team3/output/tt/c0/img_41839_keypoints.pt
Keypoints saved to /Users/rasentha/mids/w281/project/w281-fall2022-section2-team3/output/tt/c0/img_31460_keypoints.pt
Keypoints saved to /Users/rasentha/mids/w281/project/w281-fall2022-section2-team3/output/tt/c0/img_37011_keypoints.pt
Keypoints saved to /Users/rasentha/mids/w281/project/w281-fall2022-section2-team3/output/tt/c0/img_1487_keypoints.pt
Total image: 6, time taken:13.142754999999994
CPU times: user 12 s, sys: 1.11 s, total: 13.1 s
Wall time: 7.7 s


In [11]:
# import pstats
# from pstats import SortKey
# p1 = pstats.Stats(f'{config.OUTPUT_FOLDER}/mtcnn_stats')
# # p.strip_dirs().sort_stats(-1).print_stats()
# p1.sort_stats(SortKey.CUMULATIVE).print_stats(10)
# # p.sort_stats(SortKey.TIME).print_stats(10)

# p2 = pstats.Stats(f'{config.OUTPUT_FOLDER}/mtcnn_stats_model_load')
# p2.sort_stats(SortKey.CUMULATIVE).print_stats(10)


Wed Nov 30 06:47:25 2022    /Users/rasentha/mids/w281/project/w281-fall2022-section2-team3/output/mtcnn_stats

         98601988 function calls (97685069 primitive calls) in 125.128 seconds

   Ordered by: cumulative time
   List reduced from 2741 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   2545/1    0.161    0.000  125.137  125.137 {built-in method builtins.exec}
        1    0.012    0.012  125.137  125.137 <timed exec>:1(extract_poses)
        6    0.070    0.012  122.591   20.432 /var/folders/s4/p0y554f56hd_d9fk8y79dy3c0000gs/T/ipykernel_26988/826767389.py:67(get_pose)
        6    0.000    0.000  116.908   19.485 /var/folders/s4/p0y554f56hd_d9fk8y79dy3c0000gs/T/ipykernel_26988/2596138246.py:14(movenet)
        6    0.003    0.000  110.245   18.374 /Users/rasentha/opt/anaconda3/envs/amd-gpu2/lib/python3.8/site-packages/tensorflow/python/saved_model/load.py:707(load)
        6    0.000    0.000  110.243   18.374 /Users/ra

# 3. Post Processing Of Poses

In [ ]:
def process_and_save_pose(orig_pose_file, cropped_pose_file, verbose=False):
    # Read the gray scale image.
    img = cv2.imread(orig_pose_file, cv2.IMREAD_GRAYSCALE)
    PADDING = 20
    if verbose:
        plt.title('Original (grayscale)')
        plt.imshow(img, cmap='gray')
        plt.show()

    # Quick fix for removing the white border that the pose isolation code adds.
    zero_pixels = np.argwhere(img == 0)
    border_top = np.min(zero_pixels[:,0])
    border_left  = np.min(zero_pixels[:,1])
    border_bottom = np.max(zero_pixels[:,0])
    border_right = np.max(zero_pixels[:,1])
    img = img[border_top:border_bottom, border_left:border_right]
    if verbose:
        plt.title('Border removed')
        plt.imshow(img, cmap='gray')
        plt.show()

    # Find the crop region, accounting for the white border we just removed.
    positions = np.argwhere(img > 0)
    top = np.min(positions[:,0]) + border_top
    left  = np.min(positions[:,1]) + border_left
    bottom = np.max(positions[:,0]) + border_top
    right = np.max(positions[:,1]) + border_left
    d = max(right - left, bottom - top)
    padding_x = d - (bottom - top)
    padding_y = d - (right - left)
    if verbose:
        print(f'top:{top}, left:{left}, bottom:{bottom}, right:{right}, d:{d}')
    
    # Re-read the color image and apply the crop
    img = cv2.imread(orig_pose_file)
    crop_img = img[top:bottom, left:right]
    if verbose:
        plt.title('Cropped (grayscale)')
        plt.imshow(crop_img)
        plt.show()
    
    # Add padding.
    crop_img = cv2.copyMakeBorder(crop_img, PADDING, PADDING + padding_x, PADDING, padding_y + PADDING, cv2.BORDER_CONSTANT, None, 0)
    crop_img = cv2.resize(crop_img, (256, 256), interpolation = cv2.INTER_AREA)
    if verbose:
        plt.title('Padded and resized')
        plt.imshow(crop_img)
        plt.show()
    cv2.imwrite(cropped_pose_file, crop_img)
    
def post_processing(labels, pose_config, reduced=False, limit=10):
    total_images = 0
    start = time.process_time()

    for label in labels:
        if limit is not None:
            if total_images > limit:
                break
        print(f'Processing c{label} files...')
        dataset = customdataset.DriverDataset(config, reduced=reduced, label=label)
        dataloader = DataLoader(dataset, num_workers=0, batch_size=1, shuffle=False, collate_fn=dataset.get_image_from)
        for batch_idx, samples in enumerate(dataloader):
            total_images = total_images + 1
            # summary = []
            if batch_idx % 1000 == 0:
                print(f'Processed {label}:{batch_idx} files...')
                print(f'Extracted from {total_images} files...')
            if limit is not None:
                if total_images > limit:
                    break
            _, label, orig_filename = samples
            filename = os.path.basename(orig_filename)
            file_primary_name, _ = os.path.splitext(os.path.basename(filename))
            orig_pose_file = f'{pose_config.FEATURES_FOLDER_FULL}/c{label}/{file_primary_name}_pose.png'
            cropped_pose_file = f'{pose_config.FEATURES_FOLDER_FULL}/c{label}/{file_primary_name}_pose_cropped.png'            
            # Open the file and crop it to just the pose + a border of 5 pixels and then save it.
            # Resize it?
            process_and_save_pose(orig_pose_file, cropped_pose_file)
    
post_processing(config.included_labels, pose_config, limit=None)
# post_processing([0], pose_config, limit=10)
    